<a href="https://colab.research.google.com/github/deepakri201/NLSTNatureSciData/blob/main/TechnicalValidation/technicalCompliance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Technical compliance - DICOM

In this notebook, we focus on the compliance of the DICOM SEG and SR files to the DICOM standard.

We use the following tools from David Clunie to verify the files:
- dicom3tools: https://www.dclunie.com/dicom3tools.html
- dciodvfy from DICOM Toolkit: https://www.pixelmed.com/dicomtoolkit.html

Deepa Krishnaswamy

Brigham and Women's Hospital

December 2025

# Parameterization

In [33]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# General environment setup

In [34]:
!pip install idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 113.7 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


In [35]:
from idc_index import IDCClient

idc_client = IDCClient.client()

import os
import sys
import time

from google.cloud import bigquery
from google.cloud import storage

# Environment setup for dicom3tools

In [1]:
!apt-get update -qq
!apt-get install -y \
  build-essential \
  libx11-dev \
  libxt-dev \
  libmotif-dev \
  libxmu-dev \
  libtiff-dev \
  libjpeg-dev \
  libpng-dev \
  libz-dev \
  libexpat1-dev \
  wget

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'zlib1g-dev' instead of 'libz-dev'
build-essential is already the newest version (12.9ubuntu3).
libjpeg-dev is already the newest version (8c-2ubuntu10).
libjpeg-dev set to manually installed.
libexpat1-dev is already the newest version (2.4.7-1ubuntu0.6).
libexpat1-dev set to manually installed.
libpng-dev is already the newest version (1.6.37-3ubuntu0.1).
libtiff-dev is already the newest version (4.3.0-6ubuntu0.12).
libtiff-dev set to manually installed.
libx11-dev is already the newest version (2:1.7.5-1ubuntu0.3).
libx11-dev set to manually installed.
wget is already the newest version (1.21.2-2ubuntu1.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev

In [24]:
!apt-get update -qq
!apt-get install -y xutils-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  xutils-dev
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 244 kB of archives.
After this operation, 1,525 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 xutils-dev amd64 1:7.7+6ubuntu1 [244 kB]
Fetched 244 kB in 0s (1,147 kB/s)
Selecting previously unselected package xutils-dev.
(Reading database ... 118886 files and directories currently installed.)
Preparing to unpack .../xutils-dev_1%3a7.7+6ubuntu1_amd64.deb ...
Unpacking xutils-dev (1:7.7+6ubuntu1) ...
Setting up xutils-dev (1:7.7+6ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...


In [10]:
!wget https://www.dclunie.com/dicom3tools/workinprogress/dicom3tools_1.00.snapshot.20251020111927.tar.bz2

--2025-12-16 21:49:15--  https://www.dclunie.com/dicom3tools/workinprogress/dicom3tools_1.00.snapshot.20251020111927.tar.bz2
Resolving www.dclunie.com (www.dclunie.com)... 18.172.134.19, 18.172.134.75, 18.172.134.80, ...
Connecting to www.dclunie.com (www.dclunie.com)|18.172.134.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1073544 (1.0M) [application/x-bzip2]
Saving to: ‘dicom3tools_1.00.snapshot.20251020111927.tar.bz2’

dicom3tools_1.00.sn 100%[===================>]   1.02M  --.-KB/s    in 0.06s   

2025-12-16 21:49:16 (17.9 MB/s) - ‘dicom3tools_1.00.snapshot.20251020111927.tar.bz2’ saved [1073544/1073544]



In [19]:
!tar -xjf "/content/dicom3tools_1.00.snapshot.20251020111927.tar.bz2" -C "/content/"

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [28]:
%cd "/content/dicom3tools_1.00.snapshot.20251020111927"
!./Configure
# !make -j$(nproc)

/content/dicom3tools_1.00.snapshot.20251020111927
Configure: begin
Configure: we are on OS Linux
Configure: we are on instruction set architecture x86_64
Configure: make install use platform specific directory linux.6.6.105.x8664
Configure: using nawk
Configure: using bzip2 bzip2, a block-sorting file compressor.  Version 1.0.8, 13-Jul-2019.
Configure: will try to use gcc
Configure: using gcc version gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~22.04.2) 
Configure: using g++ version gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~22.04.2)  (11.4.0)
Configure: we are on Linux


In [30]:
!which imake

/usr/bin/imake


In [29]:
!imake -I./config

In [26]:
!make World

Streaming output truncated to the last 5000 lines.
	not in ../../.././libsrc/include/dconvert/bits/posix1_lim.h
	not in /usr/include/bits/posix1_lim.h
makedepend: warning:  gendump.cc (reading /usr/include/limits.h, line 199): cannot find include file "bits/posix2_lim.h"
	not in ../../.././libsrc/include/generic/bits/posix2_lim.h
	not in ../../.././libsrc/include/locale/bits/posix2_lim.h
	not in ../../.././libsrc/include/dctool/bits/posix2_lim.h
	not in ../../.././libsrc/include/pixeldat/bits/posix2_lim.h
	not in ../../.././libsrc/include/dconvert/bits/posix2_lim.h
	not in /usr/include/bits/posix2_lim.h
makedepend: warning:  gendump.cc (reading /usr/include/time.h, line 29): cannot find include file "stddef.h"
	not in ../../.././libsrc/include/generic/stddef.h
	not in ../../.././libsrc/include/locale/stddef.h
	not in ../../.././libsrc/include/dctool/stddef.h
	not in ../../.././libsrc/include/pixeldat/stddef.h
	not in ../../.././libsrc/include/dconvert/stddef.h
	not in /usr/include/stdd

In [31]:
!find . -type f -name dciodvfy

./appsrc/dcfile/dciodvfy


In [ ]:
# os.environ["PATH"] += ":/content/dicom3tools_1.00.snapshot.20251020111927/bin"

In [27]:
# !which dciodvfy
# !dciodvfy -version

/bin/bash: line 1: dciodvfy: command not found


In [32]:
!./appsrc/dcfile/dciodvfy -version

dicom3tools Version: 1.00.snapshot.20251020111927
dicom3tools Platform: Linux c898ab6b1f52 6.6.105+ #1 SMP Thu Oct  2 10:42:05 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
dicom3tools Preprocessor: gcc cpp 11.4.0
dicom3tools Compiler: gcc g++ 11.4.0
dicom3tools UID Root: 0.0.0.0


# Environment set up for DICOMSRValidator

Install java

In [51]:
!apt install openjdk-8-jdk-headless

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 2 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxtst6 amd64 2:1.2.3-1build4 [13.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 openjdk-8-jre-headless amd64 8u472-ga-1~22.04 [30.8 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 openjdk-8-jdk-headless amd64 8u472-ga-1~22.04 [8,860 kB]
Fetched 39.7

Set environment variable

In [52]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Download binaries

In [54]:
!wget -O  "/content/pixelmedjavadicom_binaryrelease.20251203.tar.bz2" https://www.dclunie.com/pixelmed/software/current/pixelmedjavadicom_binaryrelease.20251203.tar.bz2
!wget -O  "/content/pixelmedjavadicom_dependencyrelease.20251203.tar.bz2" https://www.dclunie.com/pixelmed/software/current/pixelmedjavadicom_dependencyrelease.20251203.tar.bz2

!bunzip2  '/content/pixelmedjavadicom_binaryrelease.20251203.tar.bz2'
!bunzip2 '/content/pixelmedjavadicom_dependencyrelease.20251203.tar.bz2'

--2025-12-16 23:11:06--  https://www.dclunie.com/pixelmed/software/current/pixelmedjavadicom_binaryrelease.20251203.tar.bz2
Resolving www.dclunie.com (www.dclunie.com)... 18.172.134.80, 18.172.134.19, 18.172.134.31, ...
Connecting to www.dclunie.com (www.dclunie.com)|18.172.134.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3575973 (3.4M) [application/x-bzip2]
Saving to: ‘/content/pixelmedjavadicom_binaryrelease.20251203.tar.bz2’

/content/pixelmedja 100%[===================>]   3.41M  --.-KB/s    in 0.07s   

2025-12-16 23:11:07 (45.5 MB/s) - ‘/content/pixelmedjavadicom_binaryrelease.20251203.tar.bz2’ saved [3575973/3575973]

--2025-12-16 23:11:07--  https://www.dclunie.com/pixelmed/software/current/pixelmedjavadicom_dependencyrelease.20251203.tar.bz2
Resolving www.dclunie.com (www.dclunie.com)... 18.172.134.80, 18.172.134.19, 18.172.134.31, ...
Connecting to www.dclunie.com (www.dclunie.com)|18.172.134.80|:443... connected.
HTTP request sent, awaiting re

Unzip the binaries and dependencies

In [55]:
if not os.path.isdir('/content/binaries'):
  os.mkdir('/content/binaries')
if not os.path.isdir('/content/dependencies'):
  os.mkdir('/content/dependencies')
!tar -xvf '/content/pixelmedjavadicom_binaryrelease.20251203.tar' -C '/content/binaries'
!tar -xvf '/content/pixelmedjavadicom_dependencyrelease.20251203.tar' -C '/content/dependencies'


pixelmed.jar
BUILDDATE
COPYRIGHT
DeidentifyAndRedact.bat
DeidentifyAndRedact.sh
DeidentifyAndRedactWithOriginalFileName.bat
DicomAttributeBrowser.sh
DicomBrowser.sh
DicomCleanerAssumingJREInstalled.bat
DicomCleanerWithOwnJRE.bat
DicomImageViewer.bat
DicomImageViewer.sh
DicomImageViewerWithCDJRE.bat
DicomImageViewerWithOwnJRE.bat
DicomInstanceValidator.sh
DicomSRValidator.bat
DicomSRValidator.sh
DoseUtilityAssumingJREInstalled.bat
DoseUtilityWithOwnJRE.bat
ECGViewer.bat
ECGViewer.sh
MoveDicomFilesIntoHierarchy.sh
NetworkMediaImporter.bat
NetworkMediaImporter.sh
StructuredReportBrowser.sh
StudyReceiver.bat
studyreceiver.properties
Makefile
Makefile.common.mk
README
sample.com.pixelmed.display.DicomImageViewer.properties
lib/additional/pixelmed_codec.jar
lib/additional/pixelmed_imageio.jar
./lib/additional/commons-codec-1.3.jar
lib/additional/commons-net-ftp-2.0.jar
./lib/additional/commons-compress-1.12.jar
lib/additional/hsqldb.jar
lib/additional/vecmath1.2-1.14.jar
lib/additional/jmdns

In [56]:
!tar -xvf '/content/pixelmedjavadicom_binaryrelease.20251203.tar' -C '/content/binaries'

pixelmed.jar
BUILDDATE
COPYRIGHT
DeidentifyAndRedact.bat
DeidentifyAndRedact.sh
DeidentifyAndRedactWithOriginalFileName.bat
DicomAttributeBrowser.sh
DicomBrowser.sh
DicomCleanerAssumingJREInstalled.bat
DicomCleanerWithOwnJRE.bat
DicomImageViewer.bat
DicomImageViewer.sh
DicomImageViewerWithCDJRE.bat
DicomImageViewerWithOwnJRE.bat
DicomInstanceValidator.sh
DicomSRValidator.bat
DicomSRValidator.sh
DoseUtilityAssumingJREInstalled.bat
DoseUtilityWithOwnJRE.bat
ECGViewer.bat
ECGViewer.sh
MoveDicomFilesIntoHierarchy.sh
NetworkMediaImporter.bat
NetworkMediaImporter.sh
StructuredReportBrowser.sh
StudyReceiver.bat
studyreceiver.properties
Makefile
Makefile.common.mk
README
sample.com.pixelmed.display.DicomImageViewer.properties


Copy the dependencies

In [57]:
if not os.path.isdir('/content/binaries/lib'):
  os.mkdir('/content/binaries/lib')
!cp -r /content/dependencies/lib/* /content/binaries/lib

Test command to make sure it works

In [58]:
%cd /content/binaries/
!pwd
!java -Xmx512m -Xms512m -cp "./pixelmed.jar:./lib/additional/hsqldb.jar:./lib/additional/excalibur-bzip2-1.0.jar:./lib/additional/vecmath1.2-1.14.jar:./lib/additional/jmdns.jar:./lib/additional/commons-codec-1.3.jar:./lib/additional/jai_imageio.jar" com.pixelmed.dicom.MultiFrameImageFactory

/content/binaries
/content/binaries
Error: Incorrect number of arguments
Usage: MultiFrameImageFactory inputPaths outputPath


# Download sample files

In [39]:
# We download the following sample files:
#   1. SEG file from NLSTSeg
#   2. SR features file from NLSTSeg
#   3. SR file from NLSTSybil that holds bounding box annotations

client_bq = bigquery.Client(project=project_name)

# Query to pick a series that has a SEG file from NLSTSeg and a SR bbox from NLST Sybil
query = f"""
      WITH nlstseg AS (
        SELECT DISTINCT
          PatientID,
          StudyInstanceUID,
          SeriesInstanceUID,
          ReferencedSeriesSequence[SAFE_OFFSET(0)].SeriesInstanceUID AS ReferencedSeriesInstanceUID,
          series_gcs_url
        FROM
          `bigquery-public-data.idc_v23.dicom_all`
        WHERE
          analysis_result_id = 'NLSTSeg' AND
          Modality = 'SEG'
      )
      SELECT DISTINCT
        dicom_all.PatientID,
        dicom_all.StudyInstanceUID,
        dicom_all.SeriesInstanceUID AS NLSTSybil_SeriesInstanceUID,
        nlstseg.SeriesInstanceUID AS NLSTSeg_SeriesInstanceUID,
        nlstseg.ReferencedSeriesInstanceUID AS CT_SeriesInstanceUID
      FROM
        `bigquery-public-data.idc_v23.dicom_all` as dicom_all
      JOIN
        nlstseg
      ON
        nlstseg.ReferencedSeriesInstanceUID = dicom_all.CurrentRequestedProcedureEvidenceSequence[OFFSET(0)].ReferencedSeriesSequence[OFFSET(0)].SeriesInstanceUID
      WHERE
        analysis_result_id = 'NLST-Sybil' AND
        Modality = 'SR'
      ORDER BY
        dicom_all.PatientID,
        dicom_all.StudyInstanceUID
      """
df_seg_and_sr = client_bq.query(query).to_dataframe()
df_seg_and_sr.head()

,PatientID,StudyInstanceUID,NLSTSybil_SeriesInstanceUID,NLSTSeg_SeriesInstanceUID,CT_SeriesInstanceUID
0,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.826.0.1.3680043.8.498.80137012981695130585...,1.2.276.0.7230010.3.1.3.481037312.9241.1761239...,1.2.840.113654.2.55.15708941008648745210499888...
1,100158,1.2.840.113654.2.55.81185422866512279860334872...,1.2.826.0.1.3680043.8.498.39381921446583412683...,1.2.276.0.7230010.3.1.3.481037312.10206.176123...,1.2.840.113654.2.55.31060976780967844152296392...
2,100242,1.2.840.113654.2.55.22835224307907880875083018...,1.2.826.0.1.3680043.8.498.53651851111844704842...,1.2.276.0.7230010.3.1.3.481037312.11172.176123...,1.2.840.113654.2.55.38995485391900019876570761...
3,100518,1.2.840.113654.2.55.36142141360017014809389003...,1.2.826.0.1.3680043.8.498.11954916967745944752...,1.2.276.0.7230010.3.1.3.481037312.13127.176123...,1.2.840.113654.2.55.81136962133262551156371928...
4,100570,1.2.840.113654.2.55.88862626250387223376057356...,1.2.826.0.1.3680043.8.498.60085925332680300418...,1.2.276.0.7230010.3.1.3.481037312.14098.176123...,1.2.840.113654.2.55.32380467633296345717423514...


In [40]:
# Pick a single patient that has a SEG and SR bbox file
CT_SeriesInstanceUID = df_seg_and_sr['CT_SeriesInstanceUID'].values[0]
NLSTSeg_SeriesInstanceUID = df_seg_and_sr['NLSTSeg_SeriesInstanceUID'].values[0]
NLSTSybil_SeriesInstanceUID = df_seg_and_sr['NLSTSybil_SeriesInstanceUID'].values[0]
print('CT_SeriesInstanceUID: ' + str(CT_SeriesInstanceUID))
print('NLSTSeg_SeriesInstanceUID: ' + str(NLSTSeg_SeriesInstanceUID))
print('NLSTSybil_SeriesInstanceUID: ' + str(NLSTSybil_SeriesInstanceUID))

CT_SeriesInstanceUID: 1.2.840.113654.2.55.157089410086487452104998883963678895145
NLSTSeg_SeriesInstanceUID: 1.2.276.0.7230010.3.1.3.481037312.9241.1761239683.605348
NLSTSybil_SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.80137012981695130585146174873222514935


In [42]:
# Query to get the two feature SR files from the same patient we picked

query = f"""
      SELECT DISTINCT
        SeriesInstanceUID
      FROM
        `bigquery-public-data.idc_v23.quantitative_measurements`
      WHERE
        sourceSegmentedSeriesUID = @CT_SeriesInstanceUID AND
        segmentationSeriesUID = @NLSTSeg_SeriesInstanceUID

      """
job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("CT_SeriesInstanceUID", "STRING", CT_SeriesInstanceUID),
                                                       bigquery.ScalarQueryParameter("NLSTSeg_SeriesInstanceUID", "STRING", NLSTSeg_SeriesInstanceUID)])
df_sr_features = client_bq.query(query, job_config=job_config).to_dataframe()

df_sr_features

,SeriesInstanceUID
0,1.2.276.0.7230010.3.1.3.481037312.9471.1761239...
1,1.2.276.0.7230010.3.1.3.481037312.9470.1761239...


In [43]:
# Set the SeriesInstanceUIDs for the two SRs

NLSTSeg_SR_SeriesInstanceUID_1 = df_sr_features['SeriesInstanceUID'].values[0]
NLSTSeg_SR_SeriesInstanceUID_2 = df_sr_features['SeriesInstanceUID'].values[1]

print('NLSTSeg_SR_SeriesInstanceUID_1: ' + str(NLSTSeg_SR_SeriesInstanceUID_1))
print('NLSTSeg_SR_SeriesInstanceUID_2: ' + str(NLSTSeg_SR_SeriesInstanceUID_2))

NLSTSeg_SR_SeriesInstanceUID_1: 1.2.276.0.7230010.3.1.3.481037312.9471.1761239694.918050
NLSTSeg_SR_SeriesInstanceUID_2: 1.2.276.0.7230010.3.1.3.481037312.9470.1761239694.686185


In [45]:
# Download the NLSTSeg SEG series
idc_client.download_from_selection(
    seriesInstanceUID=NLSTSeg_SeriesInstanceUID,
    downloadDir="/content/nlstseg/",
    dirTemplate='',
    quiet=True
)

# Download the NLST-Sybil SR series
idc_client.download_from_selection(
    seriesInstanceUID=NLSTSybil_SeriesInstanceUID,
    downloadDir="/content/nlstsybil",
    dirTemplate='',
    quiet=True
)

# Download the NLSTSeg shape and first-order features files
idc_client.download_from_selection(
    seriesInstanceUID=NLSTSeg_SR_SeriesInstanceUID_1,
    downloadDir="/content/nlstseg_sr_1",
    dirTemplate='',
    quiet=True
)
idc_client.download_from_selection(
    seriesInstanceUID=NLSTSeg_SR_SeriesInstanceUID_2,
    downloadDir="/content/nlstseg_sr_2",
    dirTemplate='',
    quiet=True
)

# Now rename the files

NLSTSeg_SEG_filename = os.path.join("/content/nlstseg", os.listdir("/content/nlstseg")[0])
NLSTSeg_SR_filename_1 = os.path.join("/content/nlstseg_sr_1", os.listdir("/content/nlstseg_sr_1")[0])
NLSTSeg_SR_filename_2 = os.path.join("/content/nlstseg_sr_2", os.listdir("/content/nlstseg_sr_2")[0])
NLSTSybil_SR_filename = os.path.join("/content/nlstsybil", os.listdir("/content/nlstsybil")[0])
print('NLSTSeg_SEG_filename: ' + str(NLSTSeg_SEG_filename))
print('NLSTSeg_SR_filename_1: ' + str(NLSTSeg_SR_filename_1))
print('NLSTSeg_SR_filename_2: ' + str(NLSTSeg_SR_filename_2))
print('NLSTSybil_SR_filename: ' + str(NLSTSybil_SR_filename))

NLSTSeg_SEG_filename: /content/nlstseg/e8b52f93-d553-48b4-91bf-f85e4ffa9716.dcm
NLSTSeg_SR_filename_1: /content/nlstseg_sr_1/8c70b962-616f-4f47-bbc7-af907a64330b.dcm
NLSTSeg_SR_filename_2: /content/nlstseg_sr_2/f49b29b1-2835-4fb2-9da0-f00678a6b857.dcm
NLSTSybil_SR_filename: /content/nlstsybil/0ce38fda-68d5-4b0c-bd9e-46503f4df831.dcm


# Validate the SEG and SR files using dciodvfy from dicom3tools

In [47]:
# Validate the NLSTSeg SEG file
!./appsrc/dcfile/dciodvfy $NLSTSeg_SEG_filename

Warning - Missing attribute or value that would be needed to build DICOMDIR - Study Time
Warning - Missing attribute or value that would be needed to build DICOMDIR - Study ID
Warning - Value dubious for this VR - (0x0070,0x0084) PN Content Creator's Name  PN [1] = <IDC> - Retired Person Name form
Segmentation


In [48]:
# Validate the NLSTSeg features SR 1 file
!./appsrc/dcfile/dciodvfy $NLSTSeg_SR_filename_1

Warning - Missing attribute or value that would be needed to build DICOMDIR - Study Time
Warning - Missing attribute or value that would be needed to build DICOMDIR - Study ID
EnhancedSR


In [49]:
# Validate the NLSTSeg features SR 2 file
!./appsrc/dcfile/dciodvfy $NLSTSeg_SR_filename_2

Warning - Missing attribute or value that would be needed to build DICOMDIR - Study Time
Warning - Missing attribute or value that would be needed to build DICOMDIR - Study ID
EnhancedSR


In [50]:
# Validate the NLSTSybil SR bbox file
!./appsrc/dcfile/dciodvfy $NLSTSybil_SR_filename

Warning - Missing attribute or value that would be needed to build DICOMDIR - Study Time
Warning - Missing attribute or value that would be needed to build DICOMDIR - Study ID
Comprehensive3DSR


# Validate the SR files using DICOMSRValidator

In [59]:
# Validate the NLSTSeg features SR 1
!bash DicomSRValidator.sh $NLSTSeg_SR_filename_1

Root Content Item has Template Identifier DCMR:1500 (Manufacturer "QIICR", Model "https://github.com/QIICR/dcmqi")
Found EnhancedSR IOD
Found Root Template TID_1500 (MeasurementReport)
Root Template Validation Complete
IOD validation complete


In [60]:
# Validate the NLSTSeg features SR 2
!bash DicomSRValidator.sh $NLSTSeg_SR_filename_2

Root Content Item has Template Identifier DCMR:1500 (Manufacturer "QIICR", Model "https://github.com/QIICR/dcmqi")
Found EnhancedSR IOD
Found Root Template TID_1500 (MeasurementReport)
Root Template Validation Complete
IOD validation complete


In [61]:
# Validate the NLSTSybil SR bbox file
!bash DicomSRValidator.sh $NLSTSybil_SR_filename

Root Content Item has Template Identifier DCMR:1500 (Manufacturer "Sybil", Model "")
Found Comprehensive3DSR IOD
Found Root Template TID_1500 (MeasurementReport)
Root Template Validation Complete
IOD validation complete
